<a href="https://colab.research.google.com/github/gosaitos/GEN-AI/blob/main/Pdf_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import os
import gradio as gr
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
import google.generativeai as genai

# ✅ Set Google Gemini API Key
GOOGLE_API_KEY = ""  # Replace with your Gemini API key
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
genai.configure(api_key=GOOGLE_API_KEY)

# ✅ Function to process PDF and create retriever
def create_pdf_retriever(pdf_path):
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    split_docs = text_splitter.split_documents(docs)

    # ✅ Use correct import for Google AI Embeddings
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

    # Store embeddings in FAISS vector store
    vectorstore = FAISS.from_documents(split_docs, embeddings)

    return vectorstore.as_retriever()

# ✅ Initialize Gemini AI LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# ✅ Function to get an answer based on the PDF
def answer_question(pdf_file, user_query):
    if pdf_file is None:
        return "Please upload a PDF first."

    pdf_path = pdf_file.name
    retriever = create_pdf_retriever(pdf_path)

    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
    response = qa_chain.run(user_query)

    return response if response else "Could not find relevant information."

# ✅ Gradio UI
iface = gr.Interface(
    fn=answer_question,
    inputs=[
        gr.File(label="Upload PDF"),
        gr.Textbox(lines=2, placeholder="Ask a question about the PDF content..."),
    ],
    outputs="text",
    title="PDF QA Chatbot (LangChain + Gemini AI + RAG)",
    description="Upload a PDF and ask questions about its content.",
    theme="compact"
)

iface.launch(share=True)  # Generates a public link


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1115: UserWarning: Cannot load compact. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/compact (Request ID: Root=1-67ee25a8-6c4d18512465620f0dd2542a;d0c03dd2-824b-4129-9463-024584a8c236)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b165efcc40ef47d5ba.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
